In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
classes = ['dry_asphalt_severe',
           'dry_asphalt_smooth',
           'dry_concrete_severe',
           'dry_concrete_smooth',
           'dry_gravel',
           'wet_asphalt_severe',
           'wet_asphalt_smooth']

batch_size = 64

num_classes =len(classes)

dataset_path = '/kaggle/input/rts-7-classes'

In [3]:
from keras.preprocessing.image import ImageDataGenerator

def generators(shape, preprocessing): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        horizontal_flip = True, 
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest',
        validation_split = 0.2,
    )

    height, width = shape

    train_dataset = imgdatagen.flow_from_directory(
        dataset_path,
        target_size = (height, width), 
        classes = classes,
        batch_size = batch_size,
        subset = 'training', 
    )

    val_dataset = imgdatagen.flow_from_directory(
        dataset_path,
        target_size = (height, width), 
        classes = classes,
        batch_size = batch_size,
        subset = 'validation'
    )
    return train_dataset, val_dataset

In [6]:
train_dataset, val_dataset = generators(
    (224, 224), preprocessing=keras.applications.resnet50.preprocess_input)

Found 47017 images belonging to 7 classes.
Found 11754 images belonging to 7 classes.


In [15]:
inputs = keras.Input(shape=(224, 224, 3))

"""Loading the ResNet50 model with pre-trained ImageNet weights
"""
resnet = keras.applications.ResNet50(weights='imagenet',pooling='avg', classes=num_classes,
                                 include_top=False, input_tensor=inputs)

for layer in resnet.layers:
    layer.trainable = False

In [17]:
model = keras.models.Sequential()
model.add(resnet)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(num_classes, activation='softmax'))

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 7)                 3591      
                                                                 
Total params: 24640391 (94.00 MB)
Trainable params: 1052679 (4.02 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])
checkpointer = keras.callbacks.ModelCheckpoint(filepath='best_weight.h5',
                                           monitor="val_accuracy",
                                           verbose=1, save_best_only=True)

In [27]:
history = model.fit(train_dataset, validation_data=val_dataset,
                    epochs=15, callbacks=[checkpointer], workers=4)

Epoch 1/15
735/735 [==============================] - ETA: 0s - loss: 0.6691 - accuracy: 0.7522
Epoch 1: val_accuracy improved from -inf to 0.54552, saving model to best_weight.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


735/735 [==============================] - 443s 594ms/step - loss: 0.6691 - accuracy: 0.7522 - val_loss: 1.3824 - val_accuracy: 0.5455
Epoch 2/15
735/735 [==============================] - ETA: 0s - loss: 0.5141 - accuracy: 0.8064
Epoch 2: val_accuracy improved from 0.54552 to 0.57640, saving model to best_weight.h5
735/735 [==============================] - 470s 636ms/step - loss: 0.5141 - accuracy: 0.8064 - val_loss: 1.2832 - val_accuracy: 0.5764
Epoch 3/15
735/735 [==============================] - ETA: 0s - loss: 0.4627 - accuracy: 0.8236
Epoch 3: val_accuracy improved from 0.57640 to 0.57648, saving model to best_weight.h5
735/735 [==============================] - 411s 557ms/step - loss: 0.4627 - accuracy: 0.8236 - val_loss: 1.3751 - val_accuracy: 0.5765
Epoch 4/15
735/735 [==============================] - ETA: 0s - loss: 0.4311 - accuracy: 0.8378
Epoch 4: val_accuracy did not improve from 0.57648
735/735 [==============================] - 408s 553ms/step - loss: 0.4311 - accura

In [28]:
model.save("rts.h5")